In [1]:
import pandas as pd
import spacy
import re

In [3]:
df = pd.read_csv(r"C:\Users\Edris\Desktop\Python Programmeri ng\Stice\skadebelopp\skade belopp\data.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              7000 non-null   int64 
 1   Id                      7000 non-null   object
 2   Domstol                 7000 non-null   object
 3   Malnummer               6489 non-null   object
 4   Domsnummer              2208 non-null   object
 5   Avgorandedatum          6588 non-null   object
 6   Referat dom             6589 non-null   object
 7   Avdelning               6586 non-null   object
 8   Rubrik                  7000 non-null   object
 9   Lagrum                  6594 non-null   object
 10  Rattsfall               3899 non-null   object
 11  Referat                 7000 non-null   object
 12  Sammanfattning          7000 non-null   object
 13  Utfall                  7000 non-null   object
 14  Sokord                  7000 non-null   object
 15  Litt

# i denna cell kommer vi att skapa meningar som berättar vad som hände i fallet, vi förbredder detta för lokal maskin ska lebela de med vår utvalda ord 

In [5]:
import pandas as pd
import spacy
import re

nlp = spacy.load("xx_ent_wiki_sm")
nlp.add_pipe("sentencizer")


def extract_belopp(text):
    pattern = r'(\d[\d\s\u00a0\u202f]*)(?=\s*(kr|kronor|sek))'
    matches = re.findall(pattern, text.lower())
    # Ta bort alla typer av mellanslag
    return [int(re.sub(r'[\s\u00a0\u202f]', '', m[0])) for m in matches if m[0]]

def clean_summary(sent, belopp):
    subj = ""
    obj = ""
    for token in sent:
        if token.dep_ in ("nsubj", "nsubj:pass") and not subj:
            subj = token.text
        elif token.dep_ in ("dobj", "obj", "iobj") and not obj:
            obj = token.text
    if subj and obj:
        return f"{subj} ska betala {belopp[0]} kr till {obj}"
    elif subj:
        return f"{subj} ska betala {belopp[0]} kr"
    else:
        return sent.text.strip()

results = []
seen_ids = set()

# Lägg till felhantering för debugging
for idx, (_, row) in enumerate(df.iterrows()):
    try:
        all_text = ' '.join(str(row[col]) for col in df.columns)
        doc = nlp(all_text)
        for sent in doc.sents:
            if len(sent.text.split()) > 40:
                continue
            text = sent.text.lower()
            if any(term in text for term in ['kr', 'sek', 'skadebelopp']):
                belopp = extract_belopp(sent.text)
                if belopp:
                    summary = clean_summary(sent, belopp)
                    uid = f"{row['Id']}_{summary}"
                    if uid not in seen_ids:
                        seen_ids.add(uid)
                        results.append({
                            "id": row["Id"],
                            "rubrik": row["Rubrik"],
                            "utfall": row["Utfall"],
                            "kort_mening": summary,
                            "belopp": belopp
                        })
    except Exception as e:
        print(f"Fel vid rad {idx}: {e}")
        # Valfritt: Skriv ut problematisk text för att hjälpa med debugging
        if 'sent' in locals():
            print(f"Problematisk mening: {sent.text}")

print(f"Hittade {len(results)} korta, begripliga meningar.")

# Spara som CSV istället för JSON
output_df = pd.DataFrame(results)
output_df.to_csv("adjust_data_big_data.csv", index=False, encoding="utf-8")

Hittade 26680 korta, begripliga meningar.


# Nu ska vi skicka dena data till en lokla maskin för att lebela de med våra utvalda ord 

In [12]:
df = pd.read_csv("adjust_data_big_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26680 entries, 0 to 26679
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           26680 non-null  object
 1   rubrik       26680 non-null  object
 2   utfall       26680 non-null  object
 3   kort_mening  26680 non-null  object
 4   belopp       26680 non-null  object
dtypes: object(5)
memory usage: 1.0+ MB


# Det här delen beror på din dator och detta kräver mer tid 

In [16]:
# ta bara 15000 rader 
df = df.sample(n=15000, random_state=1)
df.to_csv("adjust_data_big_data_15000.csv", index=False, encoding="utf-8")
df = pd.read_csv("adjust_data_big_data_15000.csv")

In [ ]:
import pandas as pd
import subprocess
import json
import re
from tqdm import tqdm
import os

# Ordlista med relevanta juridiska termer
ord_listan = [
    # Våldsbrott
    "misshandel", "grov misshandel", "dråp", "mord", "mordförsök", "vållande till annans död",
    "olaga tvång", "frihetsberövande", "människorov", "grovt rån", "grov kvinnofridskränkning",
    "olaga hot", "ofredande", "våld mot tjänsteman", "synnerligen grov misshandel", "grov fridskränkning",
    "kroppsskada", "kroppsskada, allvarlig", "grov kroppsskada", "tortyr", "övergrepp",
    
    # Sexualbrott
    "våldtäkt", "grov våldtäkt", "sexuellt ofredande", "sexuellt utnyttjande", "koppleri",
    "barnpornografibrott", "sexuell posering", "sexuellt övergrepp mot barn", "grooming",
    "oaktsam våldtäkt", "utnyttjande av barn för sexuell posering", "sexuellt tvång",
    "köp av sexuell tjänst", "köp av sexuell handling av barn", "grovt koppleri",
    "våldtäkt mot barn", "sexuellt utnyttjande av person i beroendeställning",
    
    # Ekonomiska brott
    "bedrägeri", "grovt bedrägeri", "förskingring", "trolöshet mot huvudman", "skattebrott",
    "bokföringsbrott", "insiderbrott", "marknadsmanipulation", "penningtvätt", "häleri",
    "svindleri", "bestickning", "mutbrott", "otillbörlig marknadspåverkan", "konkursbrott",
    "oredlighet mot borgenärer", "grovt skattebrott", "grov förskingring", "grov oredlighet mot borgenärer",
    "mannamån mot borgenärer", "bristande bokföring", "osant intygande", "urkundsförfalskning",
    "grov urkundsförfalskning", "olovlig identitetsanvändning", "subventionsmissbruk", "grovt bokföringsbrott",
    "grov trolöshet mot huvudman", "tagande av muta", "givande av muta", "handel med inflytande",
    "vårdslös finansiering av mutbrott", "skattetillägg", "kontrolluppgifter", "grovt skattebolag",
    
    # Egendomsbrott
    "stöld", "grov stöld", "snatteri", "inbrott", "rån", "grov skadegörelse",
    "skadegörelse", "självtäkt", "olovligt förfogande", "egenmäktigt förfarande",
    "tillgrepp av fortskaffningsmedel", "ringa stöld", "grovt rån", "tillgreppsbrott",
    "bostadsinbrott", "avtalsbrott", "kontraktsbrott", "besittningsintrång", "utpressning",
    "grov utpressning", "ocker", "försäkringsbedrägeri", "avtalsbrott", "dobbleri",
    
    # Trafikbrott
    "rattfylleri", "grovt rattfylleri", "vårdslöshet i trafik", "olovlig körning", "smitning",
    "hastighetsöverträdelse", "trafikbrott", "drograttfylleri", "grov vårdslöshet i trafik",
    "efterföljande rattfylleri", "fortkörning", "trafikförseelse", "fordonsförseelse", 
    "vållande till kroppsskada", "vållande till annans död i trafik", "olovlig körning, grov",
    
    # Frihets- och ordningsbrott
    "olaga intrång", "våldsamt motstånd", "uppvigling", "hets mot folkgrupp", "förtal",
    "förolämpning", "mened", "skyddande av brottsling", "övergrepp i rättssak", "ringa narkotikabrott",
    "grovt narkotikabrott", "narkotikainnehav", "dopingbrott", "olaga hot", "olaga tvång",
    "olaga förföljelse", "stalkning", "hot mot tjänsteman", "upplopp", "våldsamt upplopp",
    "störande av allmän ordning", "falsk tillvitelse", "falskt åtal", "förtigande av urkund",
    "urkundsförfalskning", "missbruk av urkund", "osant intygande", "grov mened",
    
    # Immaterialrättsliga brott
    "upphovsrättsintrång", "varumärkesintrång", "patentintrång", "mönsterskyddsintrång",
    "företagshemlighet", "upphovsrättsbrott", "piratkopiering", "intrång i upphovsrätt",
    "intrång i varumärkesrätt", "intrång i mönsterrätt", "intrång i patenträtt",
    
    # Process- och straffrätt
    "preskription", "påföljd", "straffskala", "straffvärde", "villkorlig dom", "skyddstillsyn",
    "övervakning", "samhällstjänst", "ungdomstjänst", "ungdomsvård", "rättspsykiatrisk vård",
    "fängelse", "livstids fängelse", "böter", "dagsböter", "penningböter", "förverkande",
    "kvarstad", "beslag", "husrannsakan", "avräkning", "åtalsunderlåtelse", "strafföreläggande",
    "nåd", "påföljdseftergift", "samtycke", "nödvärn", "nöd", "förmildrande omständigheter",
    "försvårande omständigheter", "medverkan till brott", "medgärningsmannaskap", "försök till brott",
    "förberedelse till brott", "stämpling till brott", "underlåtenhet", "uppsåt", "oaktsamhet",
    "grov oaktsamhet", "dolusläran", "företagsbot", "särskild rättsverkan", "brottsbalk", "vitesföreläggande",
    
    # Skadeståndsrätt
    "skadestånd", "ersättning", "personskada", "sakskada", "ren förmögenhetsskada", "kränkning",
    "sveda och värk", "lyte och men", "inkomstförlust", "olägenheter i övrigt", "skadeståndsyrkande",
    "jämkning", "solidariskt ansvar", "skadeståndsskyldighet", "ersättningsgill", "ersättningsbar",
    "skadelidande", "skadevållare", "culpa", "adekvat kausalitet", "normskydd", "förutsebar",
    "orsakssamband", "bevisbörda", "bevislättnad", "kausalitet", "culpabedömning", "presumtionsansvar",
    "strikt ansvar", "principalansvar", "produktansvar", "läkemedelsskada", "miljöskada",
    "anonyma fel", "kumulerade fel", "objektiva förutsättningar", "subjektiva förutsättningar",
    "ekonomisk skada", "ideell skada", "integritetskränkning", "kränkningsersättning",
    
    # Försäkringsrätt
    "försäkringsersättning", "ansvarsförsäkring", "trafikförsäkring", "patientförsäkring",
    "olycksfallsförsäkring", "arbetsskadeförsäkring", "trygghetsförsäkring", "brottsskadeersättning",
    "socialförsäkring", "självrisk", "regressrätt", "försäkringsfall", "försäkringsvillkor",
    "försäkringsbedrageri", "dubbelförsäkring", "överförsäkring", "underförsäkring", "förmånstagarförordnande",
    
    # Civilrätt och avtalsrätt
    "avtalsbrott", "köplag", "konsumentköp", "kontraktsbrott", "dröjsmål", "fel i vara",
    "hävning", "prisavdrag", "fullgörelse", "konventionalvite", "standardavtal", "oskäliga avtalsvillkor",
    "rättelse", "omsorgsskyldighet", "lojalitetsplikt", "klander", "ogiltighet", "jämkning av avtal",
    "avtalsvillkor", "borgen", "kvittning", "dold felansvar", "faktiskt fel", "rättsligt fel", "rådighetsfel", 
    
    # Processrätt
    "tvistemål", "brottmål", "talan", "stämningsansökan", "yrkande", "bestridande", "medgivande",
    "förkast", "bifaller", "ändring", "upphävande", "återförvisning", "rättegångskostnader",
    "ombudsarvode", "vittnesersättning", "anhålla", "häkta", "reseförbud", "anmälningsplikt",
    "offentlig försvarare", "målsägandebiträde", "rättsskydd", "rättshjälp", "rättegångsbiträde",
    "rättslig hjälp", "förhör", "editionsföreläggande", "vittnesed", "sanningsförsäkran",
    "bevisbörda", "beviskrav", "bevisning", "bevisvärdering", "bevisövervikt", "ställt utom rimligt tvivel",
    "sannolika skäl", "åtal", "åtalsplikt", "åtalspreskription", "resning", "klagan över domvilla",
    "rättskraft", "deldom", "mellandom", "tredskodom", "fullföljd av talan", "överklagande",
    "prövningstillstånd", "utevarodom"
]

def extract_keywords_with_llm(text, ordlista):
    """Använd LLM för att extrahera relevanta nyckelord från text"""
    
    prompt = f"""Analysera följande text och identifiera vilka ord från ordlistan som förekommer. 
    
    Text: {text}
    
    Ordlista: {', '.join(ordlista)}
    
    Instruktioner:
    1. Hitta alla ord från ordlistan som förekommer i texten
    2. Om du hittar flera varianter (t.ex. "misshandel" och "grov misshandel"), inkludera båda
    3. Svara endast med en lista av matchade ord separerade med kommatecken
    4. Om inga ord matchar, svara "inga matchningar"
    
    Svar:"""
    
    try:
        result = subprocess.run(
            ["ollama", "run", "llama3.1:latest"],
            input=prompt,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            encoding='utf-8',
            timeout=30
        )
        
        if result.returncode == 0:
            response = result.stdout.strip()
            if response.lower() == "inga matchningar":
                return []
            else:
                # Rensa och splitta svaret
                keywords = [kw.strip() for kw in response.split(',') if kw.strip()]
                # Validera att alla keywords finns i ordlistan
                valid_keywords = [kw for kw in keywords if kw.lower() in [word.lower() for word in ordlista]]
                return valid_keywords
        else:
            print(f"Fel från LLM: {result.stderr}")
            return []
            
    except subprocess.TimeoutExpired:
        print("LLM request timeout")
        return []
    except Exception as e:
        print(f"Fel vid LLM-anrop: {e}")
        return []

def extract_number_of_victims(text):
    """Extrahera antal offer från text med LLM"""
    
    prompt = f"""Analysera följande juridiska text och identifiera antal offer/målsägande.
    
    Text: {text}
    
    Instruktioner:
    1. Hitta antal personer som blev offer/målsägande i fallet
    2. Svara endast med ett tal (siffra)
    3. Om ingen information finns, svara "0"
    
    Svar:"""
    
    try:
        result = subprocess.run(
            ["ollama", "run", "llama3.1:latest"],
            input=prompt,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            encoding='utf-8',
            timeout=30
        )
        
        if result.returncode == 0:
            response = result.stdout.strip()
            # Extrahera första talet från svaret
            import re
            numbers = re.findall(r'\d+', response)
            if numbers:
                return numbers[0]
            else:
                return "0"
        else:
            return "0"
            
    except Exception:
        return "0"

def extract_belopp(text):
    """Extrahera skadebelopp från text med LLM"""
    
    prompt = f"""Analysera följande juridiska text och identifiera skadebeloppet/böter/ersättningen i kronor.
    
    Text: {text}
    
    Instruktioner:
    1. Hitta belopp som nämns (böter, skadeersättning, etc.)
    2. Svara endast med talet i kronor (ange siffror utan "kr")
    3. Om flera belopp finns, välj det högsta
    4. Om inget belopp finns, svara "0"
    
    Svar:"""
    
    try:
        result = subprocess.run(
            ["ollama", "run", "llama3.1:latest"],
            input=prompt,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            encoding='utf-8',
            timeout=30
        )
        
        if result.returncode == 0:
            response = result.stdout.strip()
            # Extrahera största talet från svaret
            import re
            numbers = re.findall(r'\d+', response.replace(' ', '').replace('\xa0', ''))
            if numbers:
                # Ta det största beloppet
                return str(max([int(n) for n in numbers]))
            else:
                return "0"
        else:
            return "0"
            
    except Exception:
        return "0"

def process_csv_file(csv_path, output_path):
    """Huvudfunktion för att processa CSV-filen"""
    
    # Läs CSV-filen
    try:
        df = pd.read_csv(csv_path, encoding='utf-8')
        print(f"Läst {len(df)} rader från CSV-filen")
    except Exception as e:
        print(f"Fel vid läsning av CSV: {e}")
        return
    
    # Skapa ny DataFrame med bara de kolumner du vill ha
    result_df = pd.DataFrame()
    result_df['id'] = df['id']
    result_df['special_words'] = ''
    result_df['antal_offer'] = ''
    result_df['belopp'] = ''
    
    # Processera varje rad
    print("\nProcessar rader...")
    for index, row in tqdm(df.iterrows(), total=len(df)):
        # Kombinera text från alla kolumner
        all_text = ' '.join([
            str(row['Id']), 
            str(row['belopp']), 
            str(row['utfall']), 
            str(row['rubrik']),
            str(row['kort_mening'])
        ])
        
        # Extrahera nyckelord
        keywords = extract_keywords_with_llm(all_text, ord_listan)
        result_df.at[index, 'special_words'] = ', '.join(keywords)
        
        # Extrahera antal offer
        antal = extract_number_of_victims(all_text)
        result_df.at[index, 'antal_offer'] = antal
        
        # Extrahera belopp
        belopp = extract_belopp(all_text)
        result_df.at[index, 'belopp'] = belopp
        
        # Visa progress
        if index % 5 == 0 or index == len(df) - 1:
            print(f"\nProcessat {index + 1}/{len(df)} rader")
            print(f"ID: {row['id']}")  # Using lowercase 'id' to match DataFrame column name
            print(f"Nyckelord: {', '.join(keywords) if keywords else 'Inga'}")
            print(f"Antal offer: {antal}")
            print(f"Belopp: {belopp}")
            print("-" * 40)
    
    # Skapa katalogen om den inte finns
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        print(f"Skapar katalog: {output_dir}")
        os.makedirs(output_dir)
    
    # Spara resultatet
    try:
        result_df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"\nResultat sparat till: {output_path}")
    except Exception as e:
        print(f"Fel vid sparande av fil: {e}")
        # Försök spara i samma katalog som input-filen
        backup_path = os.path.join(os.path.dirname(csv_path), 'processed_simple.csv')
        try:
            result_df.to_csv(backup_path, index=False, encoding='utf-8')
            print(f"Backup-fil sparad till: {backup_path}")
        except Exception as e2:
            print(f"Fel vid sparande av backup-fil: {e2}")
    
    # Visa slutresultat
    print("\n=== Slutresultat ===")
    print(result_df.head())
    
    # Visa statistik
    print("\n=== Statistik ===")
    print(f"Totalt antal rader: {len(result_df)}")
    print(f"Rader med nyckelord: {len(result_df[result_df['special_words'] != ''])}")
    print(f"Rader med offer: {len(result_df[result_df['antal_offer'] != '0'])}")
    print(f"Rader med belopp: {len(result_df[result_df['belopp'] != '0'])}")

def main():
    """Huvudfunktion för programmet"""
    
    # Filvägar
    input_csv = df
    output_csv = r"C:\Users\Edris\Desktop\Python Programmeri ng\Stice\skadebelopp\skade belopp\processed_simple.csv"
    
    # Kör processning
    process_csv_file(input_csv, output_csv)

if __name__ == "__main__":
    # Override input and process existing data DataFrame
    result_df = pd.DataFrame()
    result_df['id'] = df['id']
    result_df['special_words'] = ''
    result_df['antal_offer'] = ''
    result_df['belopp'] = ''
    
    # Process each row
    print("\nProcessing rows...")
    for index, row in tqdm(df.iterrows(), total=len(df)):
        # Combine text from all columns
        all_text = ' '.join([
            str(row['id']), 
            str(row['belopp']), 
            str(row['utfall']), 
            str(row['rubrik']),
            str(row['kort_mening'])
        ])
        
        # Extract keywords
        keywords = extract_keywords_with_llm(all_text, ord_listan)
        result_df.at[index, 'special_words'] = ', '.join(keywords)
        
        # Extract number of victims
        antal = extract_number_of_victims(all_text)
        result_df.at[index, 'antal_offer'] = antal
        
        # Extract amount
        belopp = extract_belopp(all_text)
        result_df.at[index, 'belopp'] = belopp

    # Save results
    result_df.to_csv('processed_simple.csv', index=False, encoding='utf-8')


Processing rows...


  0%|          | 3/15000 [00:19<27:10:31,  6.52s/it]


KeyboardInterrupt: 